# Testing the OLLAMA API

In [ ]:
%pip install requests

]11;?\Collecting requests
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (34 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (144 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import json

In [13]:
url = "http://localhost:11434/api/generate"

headers = {
    "Content-Type": "application/json"
}

data = {
    "model" : "llama3.2",
    "prompt" : "Hello, My name is Daniel Adnan",
    "stream" : False,
}

response = requests.post(url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    response_text = response.text
    data = json.loads(response_text)
    actual_response = data["response"]
    print(actual_response)
else: 
    print("Error: ", response.status_code, response.text)

Hello Daniel Adnan! It's nice to meet you. Is there something I can help you with or would you like to chat?


## Adding memory to the model

By default OLLAMA does not preserve memory

In [14]:
data = {
    "model" : "llama3.2",
    "prompt" : "What is my name?",
    "stream" : False,
}

response = requests.post(url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
    response_text = response.text
    data = json.loads(response_text)
    actual_response = data["response"]
    print(actual_response)
else: 
    print("Error: ", response.status_code, response.text)

I don't have any information about your identity, so I'm not aware of your name. This conversation just started, and we haven't established any connection or context about you. Would you like to tell me your name now?


Adding Memory

In [ ]:
%pip install ollama

1363.39s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


]11;?\Requirement already satisfied: ollama in ./.venv/lib64/python3.13/site-packages (0.4.4)
Note: you may need to restart the kernel to use updated packages.


In [7]:

from ollama import chat as ollama_chat

model = 'llama3.2'
messages = []
# Roles
USER = 'user'
ASSISTANT = 'assistant'

def add_history(content, role):
    messages.append({'role': role, 'content': content})

In [25]:
def chat(message):
    add_history(message, USER)
    response = ollama_chat(model=model, messages=messages, stream=False)
    complete_message = ''
    for line in response:
        # Check if the line is a tuple and contains the 'message' key
        if isinstance(line, tuple) and line[0] == 'message':
            message_content = line[1].content
            complete_message += message_content
            # print(message_content, end='', flush=True)
        # else:
        #     print("Unexpected line format:", line)
    add_history(complete_message, ASSISTANT)
    return complete_message

In [27]:
chat_response = chat("Hello, my name is Shadab")
print(chat_response)

I think we've reached the end of the loop! You're welcome, Shadab, for chatting with you about yourself. If you ever want to talk about something else or ask me a question, I'm here to help. Otherwise, it was great playing along with your name introduction. Have a wonderful day!


In [28]:
chat_response = chat("What is my name?")
print(chat_response)

I think we've come full circle! Your name, Shadab, has been mentioned several times throughout our conversation. If you'd like to change the subject or ask me something else, I'm here to help!


In [31]:
messages = []
chat_response = chat("What is my name?")
print(chat_response)
print(messages)
messages = []

I don't have any information about your name. I'm a conversational AI, and our conversation just started. I don't retain any personal data or knowledge about individuals. If you'd like to share your name with me, I'd be happy to chat with you!
[{'role': 'user', 'content': 'What is my name?'}, {'role': 'assistant', 'content': "I don't have any information about your name. I'm a conversational AI, and our conversation just started. I don't retain any personal data or knowledge about individuals. If you'd like to share your name with me, I'd be happy to chat with you!"}]


# Working on RAG

Getting the necessary libraries

In [ ]:
%pip install transformers datasets torch faiss-cpu matplotlib scikit-learn

]11;?\Collecting transformers
  Using cached transformers-4.47.0-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.26.5-py3-none-any.whl.metadata (13 kB)
  Using cached numpy-2.2.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux201

Add imports section

In [33]:
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
import torch
import numpy as np
import random
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np

/home/shadab/Desktop/Github Repos/RAG with OLLAMA/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
# suppress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

## Loading and preprocessing data

Import the pdf file


In [36]:

%pip install PyMuPDF

]11;?\Requirement already satisfied: PyMuPDF in ./.venv/lib64/python3.13/site-packages (1.25.1)
Note: you may need to restart the kernel to use updated packages.


In [38]:
import fitz  

# Open the PDF file
pdf_document = "ds_algo.pdf"
document = fitz.open(pdf_document)

all_text = ""

for page_num in range(len(document)):
    page = document.load_page(page_num) 
    text = page.get_text()  
    all_text += text 

print(all_text)

Data Structures and Their Algorithms: An In-depth Exploration
Data structures and their associated algorithms form the backbone of computer science and
programming. They enable efficient 
data storage, manipulation, and retrieval, making them indispensable tools in solving computational
problems. This essay 
explores various data structure algorithms, delving into their definitions, properties, usage scenarios,
and real-life applications.
Arrays are a linear data structure consisting of elements identified by an index. They store elements
of the same data type in 
contiguous memory locations. Arrays are characterized by their fixed size and ability to provide
random access to elements in 
constant time using their indices. Common algorithms associated with arrays include linear search,
binary search, bubble sort, 
and quick sort. Arrays are typically used in scenarios where data needs to be accessed directly by
index, such as in static 
tables or matrices.
A linked list is a linear dat

Process the text (splitting by paragraph)

In [40]:
# Split the text into paragraphs (simple split by newline characters)
def read_and_split_text(all_text):
    
    paragraphs = all_text.split('\n')
    paragraphs = [para.strip() for para in paragraphs if len(para.strip()) > 0]
    return paragraphs


# Split the text into paragraphs
paragraphs = read_and_split_text(all_text)

for i in range(4):
    print(f"sample: {i} paragraph: {paragraphs[i]} \n" )


sample: 0 paragraph: Data Structures and Their Algorithms: An In-depth Exploration 

sample: 1 paragraph: Data structures and their associated algorithms form the backbone of computer science and 

sample: 2 paragraph: programming. They enable efficient 

sample: 3 paragraph: data storage, manipulation, and retrieval, making them indispensable tools in solving computational 



## Embedding and indexing  

Tokenize the text

In [41]:
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')
context_tokenizer

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


DPRContextEncoderTokenizer(name_or_path='facebook/dpr-ctx_encoder-single-nq-base', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [43]:
text = paragraphs[0]
print (text)

tokens_result=context_tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=256)
tokens_result

Data Structures and Their Algorithms: An In-depth Exploration


{'input_ids': tensor([[  101,  2951,  5090,  1998,  2037, 13792,  1024,  2019,  1999,  1011,
          5995,  8993,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

Encoding into vector embeddings

In [44]:
context_encoder = DPRContextEncoder.from_pretrained('facebook/dpr-ctx_encoder-single-nq-base')

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
outputs=context_encoder(**tokens_result)
outputs

DPRContextEncoderOutput(pooler_output=tensor([[-2.0544e-01, -7.2211e-02, -1.2554e-01,  4.8373e-01,  6.7757e-02,
          2.0745e-01, -1.1438e-02, -3.9053e-01, -1.8430e-01, -3.9968e-01,
         -4.9544e-01,  2.2363e-01,  3.4093e-01,  1.4782e-01,  1.8178e-01,
          5.1610e-01,  2.1001e-01, -1.3630e-01,  1.9358e-01, -5.2526e-01,
         -3.0839e-01, -1.1096e-01,  1.7510e-01, -5.3886e-02,  1.5549e-01,
         -2.1476e-01,  2.1696e-02,  4.4649e-01, -2.0951e-01, -2.0150e-02,
         -3.0971e-01,  1.0972e-01, -1.8448e-01, -3.8305e-01,  1.6747e-01,
         -3.9037e-03, -3.0259e-01, -1.8990e-02, -3.2050e-01, -5.1606e-01,
          3.8579e-01,  5.0572e-01, -1.9355e-01,  4.4752e-01, -4.6883e-01,
         -4.2403e-01, -1.4848e+00, -2.5618e-01, -6.0209e-01, -2.1537e-01,
         -2.1714e-01,  3.3058e-01,  8.8075e-01, -2.3523e-02, -2.6663e-01,
          3.8859e-01,  3.0427e-02, -1.8447e-01,  3.1187e-01, -1.5270e-01,
          3.0220e-01, -1.1701e-01, -1.4114e-01,  1.3326e-01,  7.6226e-02,


Function to tokenize and embed the input text from PDF

In [48]:
def encode_contexts(text_list):
    embeddings = []
    for text in text_list:
        inputs = context_tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=256)
        outputs = context_encoder(**inputs)
        embeddings.append(outputs.pooler_output)
    return torch.cat(embeddings).detach().numpy()

context_embeddings = encode_contexts(paragraphs)
context_embeddings.shape

(68, 768)